# Thesis Code Part 3: ML

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import scispacy
import spacy
from negspacy.negation import Negex
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# SpaCy model for biomedical processing
nlp = spacy.load("en_core_sci_md")
nlp.add_pipe("negex")

[nltk_data] Downloading package stopwords to /home/bdrad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bdrad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Acquire Dataset

In [2]:
train_df = pd.read_csv('train_val_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

## Preprocess

In [3]:
# function from Chen & Sohn
# https://colab.research.google.com/drive/1jp8Oi2s13g2B34SPjX5074FDBlhmUdgn?usp=sharing#scrollTo=MIA9a7rckKil
def preprocess(nlp_model,input_text):
    input_text = input_text.strip()
    doc = nlp_model(input_text)
    negation_list = [0]*len(doc)
    tokens = list()
    stop = set(stopwords.words('english')+list(string.punctuation))
    stop.add("XXXX")
    
    for ent in doc.ents:
        if ent._.negex:
            index = ent.start
            while index < ent.end:
                negation_list[index] = 1
                index += 1
        
    for i,token in enumerate(doc):
        if str(token).lower() not in stop:
            if negation_list[i] == 1:
                tokens.append(("NEGEX_"+str(token).lower()))
            else:
                tokens.append(str(token).lower())

    return tokens

In [4]:
from gensim.corpora import Dictionary

train_text = train_df["Caption"].to_list()

def token_generator(text_list):
    for text in text_list:
        yield preprocess(nlp,text)

train_tokens = token_generator(train_text)
train_vocab_dict = Dictionary(train_tokens)

# gensim Dictionary does not create an id2token dictionary  until dictionary is called
train_vocab_dict[0]

'\n'

In [5]:
test_text = test_df["Caption"].to_list()
test_tokens = token_generator(test_text)

## Modeling - Naive Bayes

In [6]:
def sparse_vector_create(tuple_list,vocab_len):
    #tuple_list will have data structure akin to gensim dictionary doc2bow output 
    sparse_vector = np.zeros(vocab_len)
    for id,freq in tuple_list:
        sparse_vector[id] = freq
    return sparse_vector

def sparse_vector_generator(tokens,vocab_dict,vocab_len):
    for token in tokens:
        yield sparse_vector_create(vocab_dict.doc2bow(token),vocab_len)

# recreate generator object to reset it , otherwise will output empty result
train_tokens = token_generator(train_text)
x_train_sparse = [sparse_vector for sparse_vector in sparse_vector_generator(train_tokens,train_vocab_dict,len(train_vocab_dict))]
y_train = train_df['comm'].to_list()
y_test = test_df['comm'].to_list()

In [7]:
from sklearn import naive_bayes

nb_classifier = naive_bayes.MultinomialNB(alpha=1.0)

nb_classifier.fit(x_train_sparse,y_train)

MultinomialNB()

In [8]:
test_tokens = token_generator(test_text)
nb_predictions = list()

for token in test_tokens:
    test_sparse_vector = sparse_vector_create(train_vocab_dict.doc2bow(token),len(train_vocab_dict))
    nb_predictions.append(nb_classifier.predict(test_sparse_vector.reshape(1,-1))[0])

## Modeling - Support Vector Machine (SVM)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, smooth_idf = True,sublinear_tf = True)
tfidf.fit(x_train_sparse)
x_train_tfidf= tfidf.transform(x_train_sparse)

In [10]:
from sklearn import svm

SVM = svm.SVC(C=1.0,kernel ='linear')
SVM.fit(x_train_tfidf,y_train)

SVC(kernel='linear')

In [11]:
test_tokens = token_generator(test_text)
svm_predictions = list()

for token in test_tokens:
    test_sparse_vector = sparse_vector_create(train_vocab_dict.doc2bow(token),len(train_vocab_dict))
    x_test_tfidf = tfidf.transform(test_sparse_vector.reshape(1,-1))
    svm_predictions.append(SVM.predict(x_test_tfidf)[0])

## Evaluation

In [12]:
from sklearn import metrics

class metric_calc:
    def __init__(self, y,y_hat):
        # y is true label, y_hat is predicted label
        self.y_hat = y_hat
        self.y = y
        conf_matrix = metrics.confusion_matrix(y,y_hat)
    
        self.true_neg = conf_matrix[0][0]
        self.false_pos = conf_matrix[0][1]
        self.false_neg = conf_matrix[1][0]
        self.true_pos = conf_matrix[1][1]
        self.sum = self.true_pos + self.true_neg + self.false_pos + self.false_neg

    def conf_matrix_values(self):
        print(f"TN: {self.true_neg}, FP: {self.false_pos}, FN: {self.false_neg} TP: {self.true_pos}")
    
    def accuracy(self):
        acc = (self.true_pos + self.true_neg)/self.sum
        print(f"Accuracy is {acc:.4f}")

    def sens_spec(self):
        sens = self.true_pos/(self.true_pos+self.false_neg)
        spec = self.true_neg/(self.true_neg+self.false_pos)
        print(f"Sensitivity (aka recall) is {sens:.4f}")
        print(f"Specificity is {spec:.4f}")

    def incorrect_index(self):
        # return indices of examples that are incorrectly predicted
        index = list()
        for i,label in enumerate(self.y):
            if label != self.y_hat[i]:
                index.append(i)
        return index

In [13]:
print("Naive Bayes")
nb_metric = metric_calc(y_test,nb_predictions)
nb_metric.conf_matrix_values()
nb_metric.accuracy()
nb_metric.sens_spec()

print("SVM")
svm_metric = metric_calc(y_test,svm_predictions)
svm_metric.conf_matrix_values()
svm_metric.accuracy()
svm_metric.sens_spec()

Naive Bayes
TN: 245, FP: 7, FN: 15 TP: 96
Accuracy is 0.9394
Sensitivity (aka recall) is 0.8649
Specificity is 0.9722
SVM
TN: 250, FP: 2, FN: 4 TP: 107
Accuracy is 0.9835
Sensitivity (aka recall) is 0.9640
Specificity is 0.9921
